In [6]:
import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
from matplotlib import pyplot as plt
from multiprocessing import Pool
import time

EVENT = "FixationOnCross"  # event in behavior to align on
PRE_INTERVAL = 150   # time in ms before event
POST_INTERVAL = 350  # time in ms after event
INTERVAL_SIZE = 100  # size of interval in ms

In [7]:
SESS_BEHAVIOR_PATH = "/data/rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv"
sess_name = 20180802
behavior_path = SESS_BEHAVIOR_PATH.format(sess_name=sess_name)
beh = pd.read_csv(behavior_path)

# filter trials 
valid_beh = behavioral_utils.get_valid_trials(beh)

In [8]:
valid_beh.StimOnset

35        929682.0
36        936889.0
37        944328.0
38        951617.0
39        960217.0
           ...    
1738    13615839.0
1739    13619633.0
1740    13623543.0
1741    13627502.0
1742    13631363.0
Name: StimOnset, Length: 1707, dtype: float64

In [10]:
num_blocks_per_rule = valid_beh.groupby("CurrentRule").BlockNumber.agg('nunique')

In [14]:
valid_rules = num_blocks_per_rule[num_blocks_per_rule >= 3].index

In [16]:
valid_beh = valid_beh[valid_beh.CurrentRule.isin(valid_rules)]

In [37]:
rng = np.random.default_rng()
def get_rule_of_block(block):
    rule = block.CurrentRule.unique()[0]
    return rule
block_to_rule = valid_beh.groupby("BlockNumber").apply(get_rule_of_block).reset_index(name="CurrentBlockRule")
labels = block_to_rule.CurrentBlockRule.values.copy()
rng.shuffle(labels)
block_to_rule["ShuffledCurrentRule"] = labels


In [39]:
merged = pd.merge(valid_beh, block_to_rule, on="BlockNumber")

In [40]:
merged

,TrialNumber,BlockNumber,TrialAfterRuleChange,TaskInterrupt,ConditionNumber,Response,ItemChosen,TrialType,CurrentRule,LastRule,...,Item1Color,Item1Pattern,Item2Shape,Item2Color,Item2Pattern,Item3Shape,Item3Color,Item3Pattern,CurrentBlockRule,ShuffledCurrentRule
0,35,2,0,NaN,641,Incorrect,3.0,8,CIRCLE,CYAN,...,MAGENTA,POLKADOT,STAR,GREEN,SWIRL,SQUARE,YELLOW,ESCHER,CIRCLE,CIRCLE
1,36,2,1,NaN,627,Incorrect,3.0,8,CIRCLE,CYAN,...,YELLOW,ESCHER,STAR,GREEN,SWIRL,SQUARE,MAGENTA,POLKADOT,CIRCLE,CIRCLE
2,37,2,2,NaN,808,Incorrect,3.0,8,CIRCLE,CYAN,...,YELLOW,RIPPLE,STAR,GREEN,POLKADOT,SQUARE,CYAN,SWIRL,CIRCLE,CIRCLE
3,38,2,3,NaN,783,Incorrect,1.0,8,CIRCLE,CYAN,...,CYAN,ESCHER,STAR,GREEN,SWIRL,SQUARE,YELLOW,POLKADOT,CIRCLE,CIRCLE
4,39,2,4,NaN,1116,Incorrect,1.0,8,CIRCLE,CYAN,...,CYAN,POLKADOT,STAR,MAGENTA,RIPPLE,SQUARE,YELLOW,SWIRL,CIRCLE,CIRCLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,1738,54,31,NaN,4072,Correct,0.0,7,GREEN,CYAN,...,CYAN,ESCHER,STAR,MAGENTA,POLKADOT,CIRCLE,YELLOW,SWIRL,GREEN,MAGENTA
1599,1739,54,32,NaN,4429,Correct,0.0,7,GREEN,CYAN,...,CYAN,ESCHER,CIRCLE,MAGENTA,RIPPLE,TRIANGLE,YELLOW,POLKADOT,GREEN,MAGENTA
1600,1740,54,33,NaN,4457,Correct,0.0,7,GREEN,CYAN,...,CYAN,POLKADOT,SQUARE,MAGENTA,ESCHER,CIRCLE,YELLOW,RIPPLE,GREEN,MAGENTA
1601,1741,54,34,NaN,4380,Correct,0.0,7,GREEN,CYAN,...,CYAN,POLKADOT,TRIANGLE,MAGENTA,ESCHER,SQUARE,YELLOW,SWIRL,GREEN,MAGENTA
